# Using `kugupu` to calculate molecular coupling networks WITH ML

This notebook demonstrates how to calculate molecular coupling between fragments, inspect the results and save and load these results to file.  These results files will be the basis of all further analysis done using the `kugupu` package.

This will require version 0.20.0 of MDAnalysis, and kugupu to be installed.

In [100]:
import MDAnalysis as mda
import numpy as np
import kugupu as kgp

Firstly we create an `MDAnalysis.Universe` object from our simulation files:

In [122]:
u = mda.Universe('datafiles/C6.data', 'datafiles/C6.dcd')

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


This system has 46,500 atoms in 250 different fragments.

In [3]:
print(u.atoms.n_atoms, len(u.atoms.fragments))

46500 250


Our dynamics simulation has 5 frames of results.

In [4]:
print(u.trajectory.n_frames)

5


In [58]:
u.atoms.fragments[0].names

NoDataError: This Universe does not contain name information

To perform the coupling calculations our `Universe` will require bond information (for determining fragments) and element information (for the tight binding calculations) stored inside the `.names` attribute.

Our Lammps Data file did not include element symbols, so we can add these to the Universe now...

In [124]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.guesser.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e
        

add_names(u)

In [ ]:
from openff.toolkit.topology import Molecule
from MDAnalysis.topology.guessers import guess_types
from kugupu._yaehmop import shift_dimer_images
from MDAnalysis.lib.mdamath import triclinic_vectors


elements = guess_types(u.atoms.names)
u.add_TopologyAttr("elements", elements)
# print(u.dimensions.shape)
example_fragment = u.atoms.fragments[0]
# print(example_fragment.positions)
new_array = []
# example_fragment.positions = triclinic_vectors(example_fragment.positions, u.dimensions)

# or, for a rectangular box only:
Lx, Ly, Lz = u.dimensions[:3]
example_fragment.atoms.positions = example_fragment.positions % np.array([Lx, Ly, Lz])

# for idx, array in enumerate(example_fragment.positions):
#     array_values = array
#     if array[0] > u.dimensions[0]:
#         value_x = array_values[0] - (u.dimensions[0])
#         # assert False
#     else:
#         value_x = array_values[0]
#     if array[1] > u.dimensions[1]:
#         value_y = array_values[1] - (u.dimensions[1])
#     else:
#         value_y = array_values[1]
#     if array[2] > u.dimensions[2]:
#         value_z = array_values[2] - (u.dimensions[2])
#     else:
#         value_z = array_values[2]
#     new_array.append([value_x, value_y, value_z])

# new_array = np.array(new_array)
# example_fragment.atoms.positions = new_array

rdkit = example_fragment.convert_to.rdkit()
Molecule.from_rdkit(rdkit, allow_undefined_stereo=True)


/var/folders/8_/xls29m695yl7qglq81r94h7w0000gr/T/ipykernel_20371/1867874414.py:7: DeprecationWarning: `guess_types` is deprecated!
`guess_types` will be removed in release 3.0.0.
MDAnalysis.topology.guessers is deprecated in favour of the new Guessers API. See MDAnalysis.guesser.default_guesser for more details.
  elements = guess_types(u.atoms.names)
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:184: DeprecationWarning: `guess_atom_element` is deprecated!
`guess_atom_element` will be removed in release 3.0.0.
MDAnalysis.topology.guessers is deprecated in favour of the new Guessers API. See MDAnalysis.guesser.default_guesser for more details.
  [guess_atom_element(name) for name in atom_names], dtype=object


NGLWidget()

In [131]:
new_array

array([[1.51116638e+01, 8.20343552e+01, 4.94898376e+01],
       [1.55838165e+01, 5.22140503e-01, 4.84952393e+01],
       [1.69023666e+01, 2.65342712e-01, 4.80511322e+01],
       [1.76856308e+01, 8.15276413e+01, 4.84728241e+01],
       [1.71854248e+01, 8.06478424e+01, 4.93785095e+01],
       [1.58372650e+01, 8.08427887e+01, 4.98328400e+01],
       [1.41388245e+01, 8.22499466e+01, 4.99251404e+01],
       [1.87040710e+01, 8.13998413e+01, 4.81320343e+01],
       [1.48665924e+01, 1.75074768e+00, 4.79483643e+01],
       [1.61006393e+01, 2.22203827e+00, 4.70668945e+01],
       [1.71890869e+01, 1.35462189e+00, 4.71635895e+01],
       [1.63638840e+01, 3.37612152e+00, 4.63022461e+01],
       [1.77357407e+01, 3.41497040e+00, 4.58724823e+01],
       [1.86112595e+01, 1.98162079e+00, 4.64092102e+01],
       [1.78394775e+01, 7.94262924e+01, 4.99975739e+01],
       [1.66118774e+01, 7.89535065e+01, 5.08590088e+01],
       [1.55069504e+01, 7.97208328e+01, 5.06567230e+01],
       [1.62814484e+01, 7.79562

In [133]:
example_fragment.atoms.positions 

array([[1.51116638e+01, 8.20343552e+01, 4.94898376e+01],
       [1.55838165e+01, 5.22140503e-01, 4.84952393e+01],
       [1.69023666e+01, 2.65342712e-01, 4.80511322e+01],
       [1.76856308e+01, 8.15276413e+01, 4.84728241e+01],
       [1.71854248e+01, 8.06478424e+01, 4.93785095e+01],
       [1.58372650e+01, 8.08427887e+01, 4.98328400e+01],
       [1.41388245e+01, 8.22499466e+01, 4.99251404e+01],
       [1.87040710e+01, 8.13998413e+01, 4.81320343e+01],
       [1.48665924e+01, 1.75074768e+00, 4.79483643e+01],
       [1.61006393e+01, 2.22203827e+00, 4.70668945e+01],
       [1.71890869e+01, 1.35462189e+00, 4.71635895e+01],
       [1.63638840e+01, 3.37612152e+00, 4.63022461e+01],
       [1.77357407e+01, 3.41497040e+00, 4.58724823e+01],
       [1.86112595e+01, 1.98162079e+00, 4.64092102e+01],
       [1.78394775e+01, 7.94262924e+01, 4.99975739e+01],
       [1.66118774e+01, 7.89535065e+01, 5.08590088e+01],
       [1.55069504e+01, 7.97208328e+01, 5.06567230e+01],
       [1.62814484e+01, 7.79562

In [134]:
example_fragment.atoms.positions.shape

(186, 3)

In [37]:
u.dimensions

array([82.370605, 82.370605, 82.370605, 90.      , 90.      , 90.      ],
      dtype=float32)

In [23]:
from io import StringIO

output = StringIO()
# example_fragment.write(filename= output.getvalue(), format='.pdb')
output.name

AttributeError: '_io.StringIO' object has no attribute 'name'

In [29]:
type(example_fragment.atoms)

MDAnalysis.core.groups.AtomGroup

In [13]:
coords

array([[ 97.48227 ,  82.034355, 131.86044 ],
       [ 97.95442 ,  82.892746, 130.86584 ],
       [ 99.27297 ,  82.63595 , 130.42174 ],
       [100.05624 ,  81.52764 , 130.84343 ],
       [ 99.55603 ,  80.64784 , 131.74911 ],
       [ 98.20787 ,  80.84279 , 132.20345 ],
       [ 96.50943 ,  82.24995 , 132.29575 ],
       [101.07468 ,  81.39984 , 130.50264 ],
       [ 97.2372  ,  84.12135 , 130.31897 ],
       [ 98.471245,  84.59264 , 129.4375  ],
       [ 99.55969 ,  83.72523 , 129.5342  ],
       [ 98.73449 ,  85.74673 , 128.67285 ],
       [100.106346,  85.785576, 128.24309 ],
       [100.981865,  84.35223 , 128.77982 ],
       [100.21008 ,  79.42629 , 132.36818 ],
       [ 98.98248 ,  78.95351 , 133.22961 ],
       [ 97.877556,  79.72083 , 133.02733 ],
       [ 98.652054,  77.95627 , 134.1653  ],
       [ 97.29675 ,  77.97778 , 134.62914 ],
       [ 96.408104,  79.2198  , 133.90623 ],
       [100.44462 ,  86.95385 , 127.66395 ],
       [ 97.914085,  87.267555, 128.25804 ],
       [ 9

## Running the coupling matrix calculation

The coupling matrix between fragments is calculated using the `kgp.coupling_matrix` function.

Here we are calculating the coupling matrix for fragments in the Universe `u` where
- coupling is calculated between fragments with a closest approach of less than 5.0 Angstrom (`nn_cutoff`)
- coupling is calculated between the LUMO upwards (`state='lumo'`)
- one state per fragment is considered (`degeneracy=1`)
- we will analyse up to frame 3 (`stop=3`)

This function will (for each frame)
- identify which fragments are close enough to possibly be electronically coupled
- run a tight binding calculation between all pairs identified
- calculate the molecular coupling based on this tight binding calculation

In [ ]:
res = kgp.coupling_matrix(u, nn_cutoff=5.0, state='lumo', degeneracy=1, stop=3, model='ocelotml', client=True)

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60825 instead
  warnings.warn(
2025-06-05T16:08:09.655317+0100 INFO Processing 3 frames
2025-06-05T16:08:09.657143+0100 INFO Processing frame 1 of 3
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
2025-06-05T16:08:10.030477+0100 INFO Finding dimers within 5.0, passed 250 fragments
2025-06-05T16:08:10.387019+0100 INF

The `res` object is a namedtuple which contains all the data necessary to perform further analysis.
This object has various attributes which will not be briefly explained.

The `.frames` attribute records which frames from the trajectory were analysed.
This is useful to later cross reference data with the original MD trajectory data.

In [ ]:
print(res.frames)

[0 1 2]


The `.degeneracy` attribute stores how many degenerate states were considered for each fragment.
This value will not change over time, so this array has shape `nfragments`.

In this example only a single state per fragment was considered. 

In [ ]:
print(res.degeneracy)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


The `.H_frag` attribute contains the molecular coupling values, stored inside a 3d numpy array.
The first dimension is along the number of frames (quasi time axis),
while the other two move along fragments in the system.

For example `res.H_frag[0, 1, 71]` gives the coupling (in eV) between the 2nd and 13th fragments in the first frame.

In [ ]:
print(res.H_frag.shape)

print(res.H_frag[2, 1, 71])

(3, 250, 250)
0.0002765886942250739


Producing these results is often a time consuming part of the analysis,
therefore it is wise to save them to a file so you can come back to them later!

This can be done using the `kugupu.save_results` function, which will save the results to a hdf5 (compressed) format.

In [ ]:
kgp.save_results('myresults.hdf5', res)

These results can then be retrieved again using the `kugupu.load_results` function:

In [ ]:
kgp.load_results('./myresults.hdf5')

KugupuResults(frames=array([0, 1, 2]), H_frag=array([[[-10.27936597,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.32038834,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.35344287, ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ..., -10.43146138,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
         -10.50477574,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        , -10.37584228]],

       [[-10.38898008,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.43337746,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.44523772, ...,   0.        ,
     